# Triggers In File

Very simple code to see what triggers are in a file

In [2]:
from func_adl_servicex_xaodr25 import FuncADLQueryPHYSLITE
from servicex import deliver, ServiceXSpec, Sample, dataset
from servicex_analysis_utils import to_awk
from func_adl import ObjectStream, func_adl_callable
from func_adl_servicex_xaodr25.trigger import _add_decision_tool
import ast

from typing import TypeVar, Tuple
T = TypeVar('T')

In [3]:
from typing import List


def _tdt_fired_chains(
    s: ObjectStream[T], a: ast.Call
) -> Tuple[ObjectStream[T], ast.Call]:
    """Configure the backend to run the Trigger Decision Tool to get a list of chain

    Args:
        s (ObjectStream[T]): The stream func_adl is working on - and we can add meta data to.Tuple
        a (ast.Call): The call site in case we need to modify it

    Returns:
        Tuple[ObjectStream[T], ast.Call]: Update stream and call site.
    """
    # The TDT is very slow when it has to fetch a list of all chains in the file. Since
    # this is ServiceX, and we run on only one file at a time, we will cache the list of
    # chains in the file.
    new_s = s.MetaData(
        {
            "metadata_type": "add_cpp_function",
            "name": "tdt_chains_fired",
            "include_files": ["iostream"],
            "arguments": [],
            "code": [
                "static std::vector<std::string> all_triggers;",
                "if (all_triggers.empty()) {",
                '  all_triggers = m_trigDec->getChainGroup(".*")->getListOfTriggers();',
                "}",
                "std::vector<std::string> result;",
                "for (const auto& trigger : all_triggers) {",
                '    if (trigger.find("calratio") != std::string::npos && m_trigDec->isPassed(trigger)) {',
                "        result.push_back(trigger);",
                "    }",
                "}",
            ],
            "result_name": "result",
            "return_type": "std::vector<std::string>",
        }
    )

    return _add_decision_tool(new_s), a


@func_adl_callable(_tdt_fired_chains)
def tdt_chains_fired() -> List[str]:
    """Returns true if the event has any of the trigger chain names that have
    fired. Uses the ATLAS Trigger Decision Tool to query the event.

    Args:
        triggers (str): String specifying the triggers to check for. This is passed directly to the
        ATLAS TriggerDecisionTool, so can include any valid wildcards.

    Returns:
        bool: True if the TDT says this chain has fired on this event, false other wise.
    """
    ...

In [4]:
# Do the local deliver setup
from servicex_local import LocalXAODCodegen, WSL2ScienceImage, SXLocalAdaptor
from servicex_local import deliver as l_deliver

science = WSL2ScienceImage("atlas_al9", "25.2.82")
codegen = LocalXAODCodegen()
adaptor = SXLocalAdaptor(codegen, science, "atlasr25", "http://myserver:8080")

# Comment this next line out to use the official ServiceX service
deliver = lambda *args, **kwargs: l_deliver(*args, adaptor=adaptor, **kwargs)

# turn on INFO level logging if needed.
import logging
# logging.basicConfig(level=logging.INFO)

In [5]:
# Define base query using PHYSLITE
base_query = FuncADLQueryPHYSLITE()

# Simplified query: Select events with at least two jets with pT > 40 GeV
# Then extract jets and MET
query = (
    base_query.Select(lambda e: {
        "triggers": tdt_chains_fired(), 
    })
)

# Dataset name - using a known good PHYSLITE dataset
# ds_name = "data24_13p6TeV:data24_13p6TeV.00484979.physics_Late.deriv.DAOD_LLP1.r16740_p6815_p7079_tid47621147_00"
# ds_name = "mc23_13p6TeV:mc23_13p6TeV.562155.MGPy8EG_A14N23LO_ttbar_LL_ALP_fixed_ma40_ctau1500.deriv.DAOD_LLP1.e8599_s4369_r16083_p6876"
# ds_name = "data24_13p6TeV:data24_13p6TeV.00486211.physics_Main.deriv.DAOD_LLP1.r16518_p6815_p7079"
# ds_name = "data25_13p6TeV:data25_13p6TeV.00503591.physics_Main.deriv.DAOD_LLP1.f1615_m2272_p7079_tid47954782_00"
# ds_name = "user.gwatts:data24_LLP1_p7079_2026_02_02"
ds_name = "user.gwatts:data25_LLP1_p7079_2026_02_02"
ds_file = "../DAOD_LLP1.47621147._000005.pool.root"
ds_file = "../DAOD_LLP1.47954782._000005.pool.root.1"
# ds = dataset.Rucio(ds_name)
ds = dataset.FileList([ds_file])

# Execute the query
result = deliver(
    ServiceXSpec(
        Sample=[
            Sample(
                Name="triggers",
                Dataset=ds,
                NFiles=50,
                Query=query,
            )
        ]
    ),
)

# Convert to awkward arrays
awk_data = to_awk(result)
data = awk_data["triggers"]

In [6]:
data['triggers']

<Array [[], [], [], [], [], ..., [], [], [], []] type='10118 * var * string'>

In [7]:
import pandas as pd
from collections import Counter

# Flatten the nested arrays and count occurrences of each trigger
trigger_counts = Counter()
for event_triggers in data['triggers']:
    trigger_counts.update(event_triggers)

# Create a DataFrame with triggers and their counts
trigger_df = pd.DataFrame([
    {'Trigger': trigger, 'Event Count': count}
    for trigger, count in trigger_counts.items()
]).sort_values('Event Count', ascending=False).reset_index(drop=True)

pd.set_option("display.max_colwidth", None)
trigger_df

,Trigger,Event Count
0,HLT_j30_CLEANllp_momemfrac012_calratiovar_roiftf_preselj20emf12_L1jJ160,66
1,HLT_j30_CLEANllp_momemfrac006_calratio_L1jJ160,32
2,HLT_j30_CLEANllp_momemfrac006_calratiormbib_L1jJ160,29
3,HLT_j30_CLEANllp_momemfrac006_calratio_L1eTAU40HT,17
4,HLT_j30_CLEANllp_momemfrac006_calratio_L1eTAU60HM,17
5,HLT_j30_CLEANllp_momemfrac006_calratio_L1eTAU140,13
6,HLT_j30_CLEANllp_momemfrac006_calratiormbib_L1eTAU60HM,12
7,HLT_j30_CLEANllp_momemfrac006_calratiormbib_L1eTAU40HT,12
8,HLT_j30_CLEANllp_momemfrac006_calratiormbib_L1eTAU140,10
9,HLT_j30_CLEANllp_momemfrac006_calratio_L1eTAU80,5
